문제 1
- 배경지식

컨테이너 터미널(이하 터미널)에서는 선박이 작업할 수 있는 선석의 수가 제한되어 있다. 따라서 선박의 예상 입항 시간을 고려하여 선석 계획을 수립하여 해당 계획에 따라 작업을 진
행한다. 하지만 선박 예상 입항시간이 빈번하게 변경되고 이에 따라 선석 배정계획도 빈번하게 변경된
다. 

1. 입항 일시는 실제 입항 최소 3일전에 예상한 데이터만 의미가 있습니다. 
2. 선박의 작업 수량이 변경되는경우가 왕왕 있지만, 이부분은 고려하지 않습니다.   

- 문제

과거의 선석 배정계획의 변화와 작업 수량 별 작업 시간등을 고려하여, 
1. 어떤 선박의 예상 입항 일시가 주어졌을 때 해당 선박의 실제 입항 일시
2. 해당 선박의 작업 시간은 얼마나 걸릴지를 예상하시오
(단, 입항 일시는 실제 입항 최소 3일전에 예상한 데이터만 의미가 있으며 선박의 작업 수량
이 변경되는 경우가 있지만, 이 부분은 고려하지 않습니다)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sys
%matplotlib inline

In [7]:
# colums 인덱스 정의
data_columns = ['스케쥴 id','터미널 코드','Call Year','선박 코드','Call No','접안 일시','출발 일시','선적 작업 수량','양하 작업 수량','Shft 작업 수량','정보 수집 일시']

In [8]:
# CSV파일 불러오기
terminal_schedule_2017=pd.DataFrame(pd.read_csv('terminal_schedule_2017.csv',header=None))
terminal_schedule_2018=pd.DataFrame(pd.read_csv('terminal_schedule_2018.csv',header=None))
terminal_schedule_2019=pd.DataFrame(pd.read_csv('terminal_schedule_2019.csv',header=None))
terminal_schedule_2020=pd.DataFrame(pd.read_csv('terminal_schedule_2020.csv',header=None))
terminal_schedule_2017.columns=data_columns
terminal_schedule_2018.columns=data_columns
terminal_schedule_2019.columns=data_columns
terminal_schedule_2020.columns=data_columns

FileNotFoundError: [Errno 2] File terminal_schedule_2017.csv does not exist: 'terminal_schedule_2017.csv'

In [ ]:
# 2017~2020년 데이터 합치기
terminal_schedule = pd.concat([terminal_schedule_2017,terminal_schedule_2018,terminal_schedule_2019,terminal_schedule_2020])
terminal_schedule = terminal_schedule.reset_index()
terminal_schedule.drop('index',axis=1,inplace=True)
print(terminal_schedule)
print(terminal_schedule.dtypes)

          스케쥴 id 터미널 코드  Call Year  ... 양하 작업 수량 Shft 작업 수량             정보 수집 일시
0              1   BNCT       2017  ...    958.0       40.0                  NaN
1              2   BNCT       2017  ...    285.0       12.0                  NaN
2              3   BNCT       2017  ...    191.0        0.0                  NaN
3              4   BNCT       2017  ...    196.0        0.0                  NaN
4              5   BNCT       2017  ...      0.0        0.0                  NaN
...          ...    ...        ...  ...      ...        ...                  ...
1047759  1663231   BPTC       2020  ...    350.0        0.0  2020-11-19 01:05:22
1047760  1663232   BPTC       2020  ...    250.0        0.0  2020-11-19 01:05:22
1047761  1663233   BPTC       2020  ...    250.0        0.0  2020-11-19 01:05:22
1047762  1663234   BPTC       2020  ...      0.0        0.0  2020-11-19 01:05:22
1047763  1663235   BPTC       2020  ...      0.0        0.0  2020-11-19 01:05:22

[1047764 rows x 11 columns]

In [ ]:
# str과 float로 된 날짜 형식을 datetime으로 바꾸기

# float 형식을 str 형식으로 변환
terminal_schedule[['접안 일시','출발 일시']] = terminal_schedule[['접안 일시','출발 일시']].astype(str)
terminal_schedule['접안 일시'] = terminal_schedule['접안 일시'].str.slice(start=0,stop=14)
terminal_schedule['출발 일시'] = terminal_schedule['출발 일시'].str.slice(start=0,stop=14)

# str 형식을 datetime 형식으로 바꾸기 위한 전처리
terminal_schedule['접안 일시'] = terminal_schedule['접안 일시'].str[0:4]+'-'+terminal_schedule['접안 일시'].str[4:6]+'-'+terminal_schedule['접안 일시'].str[6:8] +' '+terminal_schedule['접안 일시'].str[8:10]+':'+terminal_schedule['접안 일시'].str[10:12]+':'+terminal_schedule['접안 일시'].str[12:14]
terminal_schedule['출발 일시'] = terminal_schedule['출발 일시'].str[0:4]+'-'+terminal_schedule['출발 일시'].str[4:6]+'-'+terminal_schedule['출발 일시'].str[6:8] +' '+terminal_schedule['출발 일시'].str[8:10]+':'+terminal_schedule['출발 일시'].str[10:12]+':'+terminal_schedule['출발 일시'].str[12:14]

# 날짜 형식이 안맞는 값 제거
terminal_schedule = terminal_schedule[terminal_schedule['접안 일시'].str.len() == 19]
terminal_schedule = terminal_schedule[terminal_schedule['출발 일시'].str.len() == 19]

# str 형식을 datetime으로 변환
terminal_schedule['접안 일시']=pd.to_datetime(terminal_schedule['접안 일시'])
terminal_schedule['출발 일시']=pd.to_datetime(terminal_schedule['출발 일시'])
terminal_schedule['정보 수집 일시']=pd.to_datetime(terminal_schedule['정보 수집 일시'])
print(terminal_schedule)
print(terminal_schedule.dtypes)

          스케쥴 id 터미널 코드  Call Year  ... 양하 작업 수량 Shft 작업 수량            정보 수집 일시
0              1   BNCT       2017  ...    958.0       40.0                 NaT
1              2   BNCT       2017  ...    285.0       12.0                 NaT
2              3   BNCT       2017  ...    191.0        0.0                 NaT
3              4   BNCT       2017  ...    196.0        0.0                 NaT
4              5   BNCT       2017  ...      0.0        0.0                 NaT
...          ...    ...        ...  ...      ...        ...                 ...
1047759  1663231   BPTC       2020  ...    350.0        0.0 2020-11-19 01:05:22
1047760  1663232   BPTC       2020  ...    250.0        0.0 2020-11-19 01:05:22
1047761  1663233   BPTC       2020  ...    250.0        0.0 2020-11-19 01:05:22
1047762  1663234   BPTC       2020  ...      0.0        0.0 2020-11-19 01:05:22
1047763  1663235   BPTC       2020  ...      0.0        0.0 2020-11-19 01:05:22

[1030601 rows x 11 columns]
스케쥴 id     

In [ ]:
#1_1 어떤 선박의 예상 입항 일시가 주어졌을 때 해당 선박의 실제 입항 일시

# 부산지역 외 다른 항구들은 제외
terminal_schedule_1_1 = terminal_schedule
terminal_schedule_1_1 = terminal_schedule_1_1.drop(terminal_schedule_1_1[(terminal_schedule_1_1['터미널 코드'] =='GWCT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='SMGT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='KITL')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='ICT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='E1CT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='HJIT')|                                                                         (terminal_schedule_1_1['터미널 코드'] =='SNCT')&
                                                                         (terminal_schedule_1_1['터미널 코드'] =='IGCT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='JUCT')|
                                                                         (terminal_schedule_1_1['터미널 코드'] =='PTCT')].index)
terminal_schedule_1_1

,스케쥴 id,터미널 코드,Call Year,선박 코드,Call No,접안 일시,출발 일시,선적 작업 수량,양하 작업 수량,Shft 작업 수량,정보 수집 일시
0,1,BNCT,2017,CMPG,004,2017-07-09 08:48:00,2017-07-10 00:25:00,872.0,958.0,40.0,NaT
1,2,BNCT,2017,CARC,003,2017-07-10 02:42:00,2017-07-10 17:50:00,947.0,285.0,12.0,NaT
2,3,BNCT,2017,IABM,019,2017-07-10 13:50:00,2017-07-10 21:50:00,23.0,191.0,0.0,NaT
3,4,BNCT,2017,TXTR,015,2017-07-10 16:10:00,2017-07-11 11:05:00,247.0,196.0,0.0,NaT
4,5,BNCT,2017,CXFZ,002,2017-07-11 06:30:00,2017-07-11 20:00:00,732.0,0.0,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1047759,1663231,BPTC,2020,NSTA,30,2020-11-21 03:00:00,2020-11-21 17:00:00,200.0,350.0,0.0,2020-11-19 01:05:22
1047760,1663232,BPTC,2020,JPPC,47,2020-11-21 05:00:00,2020-11-21 17:00:00,200.0,250.0,0.0,2020-11-19 01:05:22
1047761,1663233,BPTC,2020,NSSK,31,2020-11-23 14:00:00,2020-11-24 04:00:00,250.0,250.0,0.0,2020-11-19 01:05:22
1047762,1663234,BPTC,2020,NSSO,6,2020-12-07 06:00:00,2020-12-08 05:00:00,0.0,0.0,0.0,2020-11-19 01:05:22


1) call Number와 관계성

2) 정보수집일시 (처음과 끝의 시간차이/ 혹은 transaction 개수 

3) 1) & 2) 의 변수를 함꼐 고려

4) 월별 transaction 수?

- 정보수집일시 (transaction)이 많을수록 
2017년은 정보수집일시 column이 전부 NULL

# Call Number에 따른 실제 입항 일시

### Call Number값이 높으면 실제 입항 일시에 대한 신뢰도가 높다.

- call number가 낮다고 멀리서 오가는 배 X
- call number가 높으면 자주 오갈 수 있는 배

In [ ]:
terminal_schedule_1_1

,스케쥴 id,터미널 코드,Call Year,선박 코드,Call No,접안 일시,출발 일시,선적 작업 수량,양하 작업 수량,Shft 작업 수량,정보 수집 일시
0,1,BNCT,2017,CMPG,004,2017-07-09 08:48:00,2017-07-10 00:25:00,872.0,958.0,40.0,NaT
1,2,BNCT,2017,CARC,003,2017-07-10 02:42:00,2017-07-10 17:50:00,947.0,285.0,12.0,NaT
2,3,BNCT,2017,IABM,019,2017-07-10 13:50:00,2017-07-10 21:50:00,23.0,191.0,0.0,NaT
3,4,BNCT,2017,TXTR,015,2017-07-10 16:10:00,2017-07-11 11:05:00,247.0,196.0,0.0,NaT
4,5,BNCT,2017,CXFZ,002,2017-07-11 06:30:00,2017-07-11 20:00:00,732.0,0.0,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1047759,1663231,BPTC,2020,NSTA,30,2020-11-21 03:00:00,2020-11-21 17:00:00,200.0,350.0,0.0,2020-11-19 01:05:22
1047760,1663232,BPTC,2020,JPPC,47,2020-11-21 05:00:00,2020-11-21 17:00:00,200.0,250.0,0.0,2020-11-19 01:05:22
1047761,1663233,BPTC,2020,NSSK,31,2020-11-23 14:00:00,2020-11-24 04:00:00,250.0,250.0,0.0,2020-11-19 01:05:22
1047762,1663234,BPTC,2020,NSSO,6,2020-12-07 06:00:00,2020-12-08 05:00:00,0.0,0.0,0.0,2020-11-19 01:05:22


Call Number가 10 이상인 대상들 추리기

In [ ]:
# Call Number가 3 이하인 데이터 추리기

terminal_schedule_1_1['Call No'][0:3]
terminal_schedule_1_1

,스케쥴 id,터미널 코드,Call Year,선박 코드,Call No,접안 일시,출발 일시,선적 작업 수량,양하 작업 수량,Shft 작업 수량,정보 수집 일시
0,1,BNCT,2017,CMPG,004,2017-07-09 08:48:00,2017-07-10 00:25:00,872.0,958.0,40.0,NaT
1,2,BNCT,2017,CARC,003,2017-07-10 02:42:00,2017-07-10 17:50:00,947.0,285.0,12.0,NaT
2,3,BNCT,2017,IABM,019,2017-07-10 13:50:00,2017-07-10 21:50:00,23.0,191.0,0.0,NaT
3,4,BNCT,2017,TXTR,015,2017-07-10 16:10:00,2017-07-11 11:05:00,247.0,196.0,0.0,NaT
4,5,BNCT,2017,CXFZ,002,2017-07-11 06:30:00,2017-07-11 20:00:00,732.0,0.0,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1047759,1663231,BPTC,2020,NSTA,30,2020-11-21 03:00:00,2020-11-21 17:00:00,200.0,350.0,0.0,2020-11-19 01:05:22
1047760,1663232,BPTC,2020,JPPC,47,2020-11-21 05:00:00,2020-11-21 17:00:00,200.0,250.0,0.0,2020-11-19 01:05:22
1047761,1663233,BPTC,2020,NSSK,31,2020-11-23 14:00:00,2020-11-24 04:00:00,250.0,250.0,0.0,2020-11-19 01:05:22
1047762,1663234,BPTC,2020,NSSO,6,2020-12-07 06:00:00,2020-12-08 05:00:00,0.0,0.0,0.0,2020-11-19 01:05:22


In [ ]:
# terminal_schedule_1_2.drop(terminal_schedule_1_2[terminal_schedule_1_2['출발 일시'] >= terminal_schedule_1_2['정보 수집 일시']].index, inplace=True)
# terminal_schedule_1_2
terminal_schedule_1_1['Call No'] = pd.to_numeric(terminal_schedule_1_1['Call No'])
# terminal_schedule_1_1.drop(terminal_schedule_1_1[terminal_schedule_1_1['Call No'] < 3].index, inplace=True)

Call No가 높을 수록 예상 입항일시가 빠를 것으로 예측

# transaction 에 따른 실제 입항 일시

transaction 이 많이 일어난 데이터일수록 입항 시기가 늦다

결론 

- Call Number 높고 transaction 적은 물류 - 입항 일시 빠름
- Call Number 3 이하 transaction 많은 물류 - 입항 시기 느림

